## We demonstrate the use of LangChain with OpenAI in this file
#### To run the code here, you need to switch the Python kernel environment to where the required packages were installed or you start Jupyter-notebook from the environment. We demonstrated three operations here by using LangChain and OpenAI API.
- Instruction Tuned LLM. Here, we simply demonstrate how to call OpenAI API with our prompt and get the response.
- Instruction Tuned LLM advanced. Here, we demonstrate how to use template and customize prompt parameters using OpenAI Chat model
- Dialog Tuned LLM. Here, we will interact with OpenAI Chatbot model at advanced level

### Simple Prompting

In [1]:
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv())

True

In [ ]:
from langchain.llms import OpenAI
llm = OpenAI(model_name="text-davinci-003")
llm(prompt="explain artificial intelligence in one sentence")

### Demonstrating the usage of Chat Model

In [ ]:
#Demonstrating the usage of Chat Model
from langchain.schema import(
    AIMessage,
    HumanMessage,
    SystemMessage
)

from langchain.chat_models import ChatOpenAI

In [ ]:
chat = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.3)
messages = [
    SystemMessage(content="You are an expert in machine learning"),
    HumanMessage(content="Write a Python script to train a neural network on sample data")
    ]
response = chat(messages)
print(response)

### Demonstrating Prompt Templating. To create dynamic prompt

In [ ]:
#Demonstrating Prompt Templating. To create dynamic prompt
from langchain import PromptTemplate

template  = """
You are an expert in machine learning and you can build deep learning models. Explain the concept of {concept} in five lines.
"""

prompt = PromptTemplate(input_variables=['concept'], template=template)
prompt

In [ ]:
##Now format the prompt with the user input or data and inject it into the template
llm(prompt.format(concept="regularization")) #this can be changed to any user's input

### Demonstrating the concept of Chains

In [ ]:
#Now lets demonstrate the concept of Chains
from langchain.chains import LLMChain
chain = LLMChain(llm=llm, prompt=prompt)

#Run the chain by specifying the input variable
print(chain.run("regularization"))

In [ ]:
# We can specify mode chains. The second chain will take the output of the first chain and so on.
second_prompt = PromptTemplate(
    input_variables=["ml_concept"],
    template="Turn the concept description of {ml_concept} and explain it to me as a small kid",
)

chain_two = LLMChain(llm=llm, prompt=second_prompt)

In [ ]:
#We then combine the two chains into an overall chain
from langchain.chains import SimpleSequentialChain
overall_chain = SimpleSequentialChain(chains=[chain, chain_two], verbose=True)

#Run the chains specifying the input variable
explanation = overall_chain.run("regularization")
print(explanation)

### Demonstrating the Embedding storing using vector database

In [ ]:
#Now lets demonstrate the Embedding storing using vector database e.g PineCone. We split the text, convert it to embadding and store it in PineCone index.
#PineCone index is simply the database name. The index must be the name of pinecone index.
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 100,
    chunk_overlap = 0,
)

texts = text_splitter.create_documents([explanation])
texts

In [ ]:
#Lets check the first element of the texts documents. i.e the corpus
texts[0].page_content #the page content helps us with that

In [ ]:
#Now lets turn the texts i.e corpus to embedding. We can use OpenAI embedding to do this
from langchain.embeddings import OpenAIEmbeddings
embeddings = OpenAIEmbeddings(model_name="ada")
embeddings

In [ ]:
#lets search for the first document in our corpus i.e texts in the embedding vectors to know it works.
query_result = embeddings.embed_query(texts[0].page_content)
query_result #this will display the vector embedding

In [ ]:
#Now lets store the vector representation (i.e vector embedding) inside the PineCone index (i.e db).
import os
import pinecone
from langchain.vectorstores import Pinecone

#Initialize pinecone
pinecone.init(api_key=os.getenv['PINECONE_API_KEY'], environment=os.getenv(['PINECONE_ENV']))

In [ ]:
index_name = "langchain-test"
search = Pinecone.from_documents(texts, embeddings, index_name=index_name)

In [ ]:
#Lets do similarity_search on the PineCone index that we just created. We can search a specific document.
query="What is important about regularization?"
result = search.similarity_search(query)
result
#If you do to your PineCone account, you will see everything inside the index. In vector database, index is called the database name

### Demonstrating the concepts of Agents

In [ ]:
#Now lets demonstrate the concepts of Agents like ChatBot. Here, we will ask OpenAI ChatBot to solve a mathematical equation problem e.g quadratic
from langchain.agents.agent_toolkits import create_python_agent
from langchain.tools.python.tool import PythonREPLTool
from langchain.python import PythonREPL
from langchain.llms.openai import OpenAI

In [ ]:
#We instantiate a python agent executor
agent_executor = create_python_agent(
    llm=OpenAI(temperature=0, max_tokens=1000),
    tool=PythonREPL(),
    verbose=True
)

In [ ]:
agent_executor.run("Find the roots (zeros) if the quadratic function 6 * x**2 + 3*x -5")